In [18]:
import torch
from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential

In [29]:
x = torch.randn(128, 32)
td = TensorDict({"x": x}, batch_size=128)
A = torch.nn.Linear(32, 16)
B = torch.nn.Linear(16, 1)
net = TensorDictSequential(
    TensorDictModule(B, in_keys=['x'], out_keys=['x']),
    TensorDictModule(A, in_keys=['x'], out_keys=['x']),
)
print(td)
td['x']

TensorDict(
    fields={
        x: Tensor(shape=torch.Size([128, 32]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([128]),
    device=None,
    is_shared=False)


tensor([[-1.2597,  0.6146, -0.1494,  ..., -1.2600, -1.1794,  0.0224],
        [ 0.6222, -0.1293, -1.0904,  ...,  2.6281, -1.4608, -1.5657],
        [-0.0684,  0.3130,  0.8169,  ..., -0.3447, -0.3370, -1.9029],
        ...,
        [-1.6598, -0.3143,  1.6696,  ..., -0.3619, -0.4201,  0.9876],
        [-0.4210,  0.1313,  2.3547,  ..., -0.6206,  1.3409,  2.1076],
        [-1.0375, -0.5295,  2.5803,  ...,  0.9029, -0.2251, -0.7743]])

In [31]:
x = torch.where(x < 0, x, torch.nan)
x

tensor([[-1.2597,     nan, -0.1494,  ..., -1.2600, -1.1794,     nan],
        [    nan, -0.1293, -1.0904,  ...,     nan, -1.4608, -1.5657],
        [-0.0684,     nan,     nan,  ..., -0.3447, -0.3370, -1.9029],
        ...,
        [-1.6598, -0.3143,     nan,  ..., -0.3619, -0.4201,     nan],
        [-0.4210,     nan,     nan,  ..., -0.6206,     nan,     nan],
        [-1.0375, -0.5295,     nan,  ...,     nan, -0.2251, -0.7743]])

In [33]:
%timeit x[:, :4].isfinite()

5.31 μs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [28]:
net(td)
td

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x32 and 16x1)

In [25]:
td['x']

tensor([[-0.1361],
        [-0.1346],
        [-0.0069],
        [ 0.2074],
        [ 0.1612],
        [ 0.0645],
        [ 0.0271],
        [ 0.1730],
        [-0.0341],
        [ 0.1541],
        [ 0.1039],
        [ 0.1412],
        [ 0.0702],
        [-0.1107],
        [-0.1642],
        [ 0.0241],
        [-0.1175],
        [ 0.2938],
        [-0.2836],
        [ 0.2461],
        [ 0.0905],
        [ 0.3248],
        [ 0.0477],
        [ 0.1142],
        [ 0.1983],
        [ 0.1797],
        [ 0.5674],
        [ 0.0976],
        [ 0.4403],
        [ 0.2673],
        [ 0.3029],
        [ 0.1367],
        [ 0.0492],
        [ 0.2563],
        [ 0.5119],
        [ 0.0183],
        [ 0.0755],
        [ 0.3572],
        [ 0.1312],
        [ 0.1133],
        [ 0.0307],
        [-0.0640],
        [ 0.2943],
        [-0.1020],
        [ 0.1032],
        [ 0.2347],
        [ 0.1656],
        [-0.0146],
        [ 0.2690],
        [ 0.1730],
        [-0.1129],
        [-0.2304],
        [ 0.

In [26]:
B(A(x))

tensor([[-0.1361],
        [-0.1346],
        [-0.0069],
        [ 0.2074],
        [ 0.1612],
        [ 0.0645],
        [ 0.0271],
        [ 0.1730],
        [-0.0341],
        [ 0.1541],
        [ 0.1039],
        [ 0.1412],
        [ 0.0702],
        [-0.1107],
        [-0.1642],
        [ 0.0241],
        [-0.1175],
        [ 0.2938],
        [-0.2836],
        [ 0.2461],
        [ 0.0905],
        [ 0.3248],
        [ 0.0477],
        [ 0.1142],
        [ 0.1983],
        [ 0.1797],
        [ 0.5674],
        [ 0.0976],
        [ 0.4403],
        [ 0.2673],
        [ 0.3029],
        [ 0.1367],
        [ 0.0492],
        [ 0.2563],
        [ 0.5119],
        [ 0.0183],
        [ 0.0755],
        [ 0.3572],
        [ 0.1312],
        [ 0.1133],
        [ 0.0307],
        [-0.0640],
        [ 0.2943],
        [-0.1020],
        [ 0.1032],
        [ 0.2347],
        [ 0.1656],
        [-0.0146],
        [ 0.2690],
        [ 0.1730],
        [-0.1129],
        [-0.2304],
        [ 0.